In [2]:
import tensorflow as tf
import tensorflow.python.keras
#import keras
from tensorflow.python.keras import backend as K
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
print(tf.__version__)
import cv2  # for image processing
from sklearn.model_selection import train_test_split
import scipy.io
import os
import scipy
#from scipy.spatial.distance import directed_hausdorff, cosine
from IPython import display

2.0.0-beta1


## Utils function

**Create dataframe function**

In [3]:
def create_dataframe(path):
    abs_path = os.path.join(path, 'items.json' )
    print(abs_path)
    df_artworks = pd.read_json(abs_path)
    df_artworks = df_artworks.set_index('id')
    df_artworks = df_artworks.sort_index()
    df_artworks = df_artworks.reset_index()
    #clean data
    df_artworks['data'] = df_artworks['data'].str.replace('\r','')
    return df_artworks

**Encode artwork function**

In [4]:
from arts_preprocess_utils import get_image

def encode_artworks(x, encoder, base_img_path):

    try:
        img_path = os.path.join(base_img_path, str(x)+'.jpg')
        # Images
        image = get_image(img_path, img_Height=img_Height, img_Width=img_Width)

        # Encode
        code = encoder.predict(image)
        
        return code.reshape((-1,))

    except:
        print("Problem with file " + x)
        return None


## Load encoder

In [5]:
from tensorflow.python.keras.models import load_model

autoencoder_model = load_model('/root/work/trained_models/denoisy_encoder.h5')

## Load artwork's metadata

In [ ]:
BASE_PATH = '/root/work/datasets/artwork_sequence/'

#According to the dataset input used during autoencoder training
img_Height = 128
img_Width = 128
chanels= 3
N_CLASSES = 1

In [ ]:
#List museums
for folder in os.listdir(BASE_PATH):
    museum_path = os.path.join(BASE_PATH, folder)
    #List tours inside the museum
    for tour_folder in os.listdir(museum_path):
        try:
            #Load tour data
            tour_path = os.path.join(museum_path, tour_folder)
            df = create_dataframe(tour_path)
            #Save data in csv format
            df.to_csv(os.path.join(tour_path, 'metadata.csv' ), index=False)

            #Encode artwork's tour
            base_img_path = os.path.join(tour_path, 'images' )
            df['code'] = df['id'].apply(encode_artworks, encoder=autoencoder_model, base_img_path=base_img_path)

            #Save code matrix
            encodes_list = list(df['code'])
            encode_matrix = np.stack(encodes_list)
            np.save(os.path.join(tour_path, 'code_matrix' ), encode_matrix)
        except:
            print("Problem with folder " + tour_folder)

In [ ]:
df = pd.read_csv('/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_14/metadata.csv')
df.head()

## Load all_metadata artworks dataframe

In [6]:
BASE_PATH = '/root/work/datasets/artwork_sequence/'

#According to the dataset input used during autoencoder training
img_Height = 128
img_Width = 128
chanels= 3
N_CLASSES = 1

In [7]:
df_all_metadata = pd.read_csv(os.path.join(BASE_PATH, 'all_metadata.csv'))
df_all_metadata.head()

,id,author,data,image_url,title,tour_path
0,1,n.v. haagsche plateelfabriek rozenburg,1914,https://lh3.googleusercontent.com/IJn7rB4WvYvv...,vaas beschilderd met paarse seringen en op een...,/root/work/datasets/artwork_sequence/rijksmuse...
1,2,n.v. haagsche plateelfabriek rozenburg,1900,https://lh3.googleusercontent.com/l0ccWh5aCgP5...,vaas,/root/work/datasets/artwork_sequence/rijksmuse...
2,3,n.v. haagsche plateelfabriek rozenburg,1900,https://lh3.googleusercontent.com/uNQWFg-BhiPZ...,vase with lily decoration,/root/work/datasets/artwork_sequence/rijksmuse...
3,4,n.v. haagsche plateelfabriek rozenburg,1902,https://lh3.googleusercontent.com/QRdRjQDGyvDp...,vaas beschilderd met takken met seringen en ee...,/root/work/datasets/artwork_sequence/rijksmuse...
4,6,theo colenbrander,1886,https://lh3.googleusercontent.com/TZqVQVxb-1kl...,garniture of five vases,/root/work/datasets/artwork_sequence/rijksmuse...


In [8]:
df_all_metadata['img_path'] = df_all_metadata['tour_path'].apply(lambda x : os.path.join(x, 'images'))
df_all_metadata['img_path'].iloc[0]

'/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_21/images'

In [9]:
#Encode artwork's tour
df_all_metadata.loc[:,'code'] = df_all_metadata.apply( lambda row : encode_artworks(x= row['id'], encoder=autoencoder_model, base_img_path=row['img_path']), axis = 1) 

#Save code matrix
encodes_list = list(df_all_metadata['code'])
encode_matrix = np.stack(encodes_list)
np.save(os.path.join(BASE_PATH, 'all_code_matrix' ), encode_matrix)

In [10]:
encode_matrix.shape

(633, 300)